In [1]:
import os
import pandas as pd
import numpy as np
import torch
import datetime as dt
import torch.nn as nn
from torchdiffeq import odeint_adjoint as odeint
import csv
from pathlib import Path
from scipy import signal
from ipywidgets import interact
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm

import torch.nn.functional as F

In [2]:
def detect_icp_pulses(icp_raw, fs=100):
    icp = signal.detrend(icp_raw)
    Wc1 = 20 / fs
    Wc2 = 0.7 / (fs/2)

    b1, a1 = signal.iirfilter(N=8, Wn=Wc1, btype='lowpass', rs=60, rp=1, ftype='cheby1', output='ba')
    f_icp = signal.filtfilt(b1, a1, icp)

    b2, a2 = signal.iirfilter(N=8, Wn=Wc2, btype='highpass', rs=60, rp=1, ftype='cheby1', output='ba')
    f_icp = signal.filtfilt(b2, a2, f_icp)

    dICP = np.diff(f_icp)
    SSF = np.insert(dICP, 0, 0)
    SSF[np.argwhere(SSF < 0)] = 0

    z_icp = np.copy(f_icp)
    z_icp[np.argwhere(z_icp < 0)] = 0
    nSSF = np.multiply(SSF, z_icp)

    min_dist = 0.4 * fs
    pk_locs = signal.find_peaks(nSSF, distance=min_dist)[0]

    po_locs = []
    for pk_ind, pk_loc in enumerate(pk_locs[:-1]):
        w = f_icp[pk_locs[pk_ind]:pk_locs[pk_ind + 1]]
        w_rev = w[::-1]
        offset = np.argmin(w_rev)
        po_locs.append(pk_locs[pk_ind + 1] - offset - 1)

    return f_icp, po_locs


In [3]:
from models.SiameseModels import SiameseNeuralODE
from models.ResnetODEmodels import ODE

class_dict = {
    0: "T1",
    1: "T2",
    2: "T3",
    3: "T4",
    4: "A+E"
}

name = "Resampling_SiameseODE_5cls_2020-06-16_SiameseODE_1"
model_path = os.path.join(os.getcwd(), "experiments", name, "model_1.pth")

device = torch.device('cuda:' + str(0) if torch.cuda.is_available() else 'cpu')
model_siam = SiameseNeuralODE(len(class_dict.keys())).to(device)
model_siam.load_state_dict(torch.load(model_path, map_location=device)["state_dict"])
model_siam.eval()

name = "2020-06-11_ODE_5cls_1"
model_path = os.path.join(os.getcwd(), "experiments", name, "model_1.pth")

model_ode = ODE(len(class_dict.keys())).to(device)

state_dict = torch.load(model_path, map_location=device)["state_dict"]
new_state_dict = {}
for key in state_dict.keys():
    new_state_dict["model."+key] = state_dict[key]

model_ode.load_state_dict(new_state_dict)
model_ode.eval()

ODE(
  (model): Sequential(
    (0): Conv1d(1, 64, kernel_size=(3,), stride=(1,))
    (1): ResBlock(
      (norm1): GroupNorm(32, 64, eps=1e-05, affine=True)
      (relu): ReLU(inplace=True)
      (downsample): Conv1d(64, 64, kernel_size=(1,), stride=(2,), bias=False)
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(2,), padding=(1,), bias=False)
      (norm2): GroupNorm(32, 64, eps=1e-05, affine=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    )
    (2): ResBlock(
      (norm1): GroupNorm(32, 64, eps=1e-05, affine=True)
      (relu): ReLU(inplace=True)
      (downsample): Conv1d(64, 64, kernel_size=(1,), stride=(2,), bias=False)
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(2,), padding=(1,), bias=False)
      (norm2): GroupNorm(32, 64, eps=1e-05, affine=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    )
    (3): ODEBlock(
      (odefunc): ODEfunc(
        (norm1): GroupNor

In [4]:
path = os.path.join(os.getcwd(), "datasets", "new_patients")

def files(path, ext=None):
    for (dirpath, dirnames, filenames) in os.walk(path):
        for file in filenames:
            if os.path.isfile(os.path.join(dirpath, file)):
                f_dta = None
                for file_dta in filenames:
                    if file.split(".")[0] in file_dta and file != file_dta:
                        f_dta = file_dta
                
                if ext is None:
                    yield dirpath, file, f_dta
                elif ext in file.split(".")[-1]:
                    yield dirpath, file, f_dta

file_paths = []
for file in files(path, "pkl"):
    file_paths.append(file)

loaded = []
for file_path in file_paths:
    with open(os.path.join(file_path[0], file_path[1]), "rb") as file:
        loaded.append(pickle.load(file))

In [5]:
data = []
for patient in tqdm(loaded):
    time = patient['t']
    icp = patient['icp']
    abp = patient['abp']
    fs = 1/(time[1]-time[0])
    f_icp, po_locs = detect_icp_pulses(icp, fs)
    data.append({
        "time": time,
        "icp": icp,
        "abp": abp,
        "f_icp": f_icp,
        "po_locs": po_locs
    })
    

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:14<00:00,  4.90s/it]


In [6]:
from scipy import interpolate

def get_batches(pulses, batch_size=2048, sig_length=180):
    for i in range(0,len(pulses),batch_size):
        tensors = []
        for pulse in pulses[i:i+batch_size]:
            interp = interpolate.interp1d(np.arange(0, len(pulse), 1), pulse,
                                            kind="cubic")
            new_t = np.linspace(0, len(pulse)-1, 180)
            data = interp(new_t)
            
            data = data - np.min(data)
            
            if np.max(data) != 0:
                data = data / np.max(data)
            
            tensors.append(data)
        yield torch.tensor(tensors,dtype=torch.float)

In [7]:
import scipy.signal as sp_sig


def find_signal_offset_by_min(x, y):
    d_x = np.diff(x)
    d_x = np.insert(d_x, 0, 0)
    d_y = np.diff(y)
    d_y = np.insert(d_y, 0, 0)
    offset = np.argmax(sp_sig.correlate(d_x, d_y)) - len(d_y)
    return -offset


def shift_signal(signal, offset):
    if offset == 0:
        new_signal = signal
    else:
        new_signal = np.zeros(len(signal))
        if offset > 0:
            new_signal[:-offset] = signal[offset:]
        elif offset < 0:
            new_signal[abs(offset):] = signal[:-abs(offset)]
    return new_signal

all_icp_pulses = []
all_abp_pulses = []
predictions_siam = []
predictions_ode = []
predicted_labels_siam = []
predicted_labels_ode = []
icp_full = []
abp_full = []
    
for patient in tqdm(data):
    f_icp = patient["f_icp"]
    abp = patient["abp"]
    po_locs = patient["po_locs"]
    pulses = [f_icp[po_locs[i]: po_locs[i+1]] 
                  for i in range(len(po_locs)-1)]
    
    pulses_abp = [shift_signal(abp[po_locs[i]: po_locs[i+1]], find_signal_offset_by_min(pulses[i], abp[po_locs[i]: po_locs[i+1]]))
                  for i in range(len(po_locs)-1)]
    
    all_outputs = []
    for batch in tqdm(zip(get_batches(pulses), get_batches(pulses_abp))):
        icp_full.append(batch[0].numpy())
        abp_full.append(batch[1].numpy())
        icp = batch[0].to(device)
        abp = batch[1].to(device)
    
        prediction_siam = model_siam(icp, abp)
        predictions_siam.append(prediction_siam.detach().cpu().numpy())

        icp = icp.unsqueeze(1)
        prediction_ode = model_ode(icp)
        predictions_ode.append(prediction_ode.detach().cpu().numpy())
        predicted_siam = list(np.argmax(prediction_siam.cpu().detach().numpy(), axis=1))
        predicted_ode = list(np.argmax(prediction_ode.cpu().detach().numpy(), axis=1))
        predicted_labels_siam+=predicted_siam
        predicted_labels_ode+=predicted_ode

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:09,  9.92s/it]
2it [00:19,  9.83s/it]
3it [00:29,  9.82s/it]
4it [00:38,  9.62s/it]
5it [00:48,  9.60s/it]
6it [00:57,  9.59s/it]
7it [01:07,  9.56s/it]
8it [01:16,  9.59s/it]
9it [01:26,  9.68s/it]
10it [01:36,  9.67s/it]
11it [01:45,  9.67s/it]
12it [01:55,  9.69s/it]
13it [02:05,  9.74s/it]
14it [02:14,  9.61s/it]
15it [02:24,  9.56s/it]
16it [02:33,  9.59s/it]
17it [02:43,  9.67s/it]
18it [02:53,  9.68s/it]
19it [03:03,  9.73s/it]
20it [03:13,  9.76s/it]
21it [03:23,  9.78s/it]
22it [03:32,  9.79s/it]
23it [03:42,  9.80s/it]
24it [03:52,  9.82s/it]
25it [04:02,  9.83s/it]
26it [04:12, 10.02s/it]
27it [04:22,  9.86s/it]
28it [04:31,  9.74s/it]
29it [04:41,  9.66s/it]
30it [04:50,  9.60s/it]
31it [05:00,  9.56s/it]
32it [05:09,  9.53s/it]
33it [05:19,  9.72s/it]
34it [05:29,  9.75s/it]
35it [05:39,  9.67s/it]
36it [05:48,  9.62s/it]
37it [

In [20]:
predictions_siam = [preds for gr in predictions_siam for preds in gr]
predictions_ode = [preds for gr in predictions_ode for preds in gr]
icp_full = [preds for gr in icp_full for preds in gr]
abp_full = [preds for gr in abp_full for preds in gr]
checkpoint = {
"predictions_siam":predictions_siam,
"predictions_ode":predictions_ode,
"predicted_labels_siam":predicted_labels_siam,
"predicted_labels_ode":predicted_labels_ode,
"icp_full":icp_full,
"abp_full":abp_full
}

with open(os.path.join(os.getcwd(), "results", "new_patients_checkpoint_1.pkl"),"wb") as file:
    pickle.dump(checkpoint,file)

In [2]:
with open(os.path.join(os.getcwd(), "results", "new_patients_checkpoint_1.pkl"),"rb") as file:
    checkpoint = pickle.load(file)

In [3]:
df = pd.DataFrame(checkpoint)

In [32]:
queried = df.query("predicted_labels_siam == 0")
@interact(y=(0,len(queried),1))
def g(y):
    plt.plot(queried.iloc[y]["icp_full"])
    

interactive(children=(IntSlider(value=19380, description='y', max=38760), Output()), _dom_classes=('widget-int…

In [4]:
from random import choices
chosen = []
for i in range(5):
    queried = df.query("predicted_labels_siam == "+str(i))
    cho = choices(np.arange(len(queried)), k=110)
    chosen += [queried.iloc[ch] for ch in cho]

In [5]:
out_dict = {
"ID" : [],
"predictions_siam" : [],
"predictions_ode" : [],
"predicted_labels_siam" : [],
"predicted_labels_ode" : [],
"icp_full" : [],
"abp_full" : []
}
for choice in chosen:
    out_dict["ID"].append(choice.name)
    out_dict["predictions_siam"].append(choice["predictions_siam"])
    out_dict["predictions_ode"].append(choice["predictions_ode"])
    out_dict["predicted_labels_siam"].append(choice["predicted_labels_siam"])
    out_dict["predicted_labels_ode"].append(choice["predicted_labels_ode"])
    out_dict["icp_full"].append(choice["icp_full"])
    out_dict["abp_full"].append(choice["abp_full"])

In [6]:
df_out = pd.DataFrame(out_dict)

In [7]:
len(df_out.query("predicted_labels_siam != predicted_labels_ode"))/len(df_out)

0.3472727272727273

In [8]:
df_out.to_csv(os.path.join(os.getcwd(), "results", "additional_patients_check.csv"))

In [17]:
df_compare = pd.read_csv(os.path.join(os.getcwd(), "results", "shuffled_new_patients_check.csv"))
id_compare = df_compare["ID"]
id_new = df_out["ID"]
no_all = 0
for value in id_compare:
    if value in id_new:
        no_all+=1
print(no_all)

0


In [47]:
save_path = os.path.join(os.getcwd(), "results", "additional_experts_check")
for i in range(len(df_out)):
    item = df_out.iloc[i]
    plt.plot(item["icp_full"], label="ICP")
    plt.plot(item["abp_full"], label="ABP")
    plt.legend(loc="upper left")
    plt.savefig(os.path.join(save_path, str(i)+'.png'))
    plt.clf()

<Figure size 432x288 with 0 Axes>

In [76]:
df = pd.read_csv(os.path.join(os.getcwd(), "results", "new_patients_check.csv"))
df = df.set_index(df.columns[0])

In [18]:
import scipy.signal as sp_sig

def shuffle(df, n=1, axis=0):     
    df = df.copy()
    for _ in range(n):
        df.apply(np.random.shuffle, axis=axis)
    return df

def find_signal_offset_by_min(x, y):
    d_x = np.diff(x)
    d_x = np.insert(d_x, 0, 0)
    d_y = np.diff(y)
    d_y = np.insert(d_y, 0, 0)
    offset = np.argmax(sp_sig.correlate(d_x, d_y)) - len(d_y)
    return -offset


def shift_signal(signal, offset):
    if offset == 0:
        new_signal = signal
    else:
        new_signal = np.zeros(len(signal))
        if offset > 0:
            new_signal[:-offset] = signal[offset:]
        elif offset < 0:
            new_signal[abs(offset):] = signal[:-abs(offset)]
    return new_signal


In [21]:
df = df_out
new_df_dict = {
    "ID" : [],
    "predictions_siam" : [],
    "predictions_ode" : [],
    "predicted_labels_siam" : [],
    "predicted_labels_ode" : [],
    "icp_full" : [],
    "abp_full" : []
}
for row in df.iterrows():
    row = row[1]
    new_df_dict["ID"].append(row["ID"])
    new_df_dict["predictions_siam"].append(row["predictions_siam"])
    new_df_dict["predictions_ode"].append(row["predictions_ode"])
    new_df_dict["predicted_labels_siam"].append(row["predicted_labels_siam"])
    new_df_dict["predicted_labels_ode"].append(row["predicted_labels_ode"])
    icp = row["icp_full"]
#     icp = row["icp_full"][1:-1].split(" ")
#     icp = [float(value.strip("\n")) for value in icp if value is not ""]
    new_df_dict["icp_full"].append(icp)
    abp = row["abp_full"]
#     abp = row["abp_full"][1:-1].split(" ")
#     abp = [float(value.strip("\n")) for value in abp if value is not ""]
    icp = np.array(icp)
    abp = np.array(abp)
    offset = find_signal_offset_by_min(icp, abp)
    shift_abp = shift_signal(abp, offset)
    new_df_dict["abp_full"].append(shift_abp)

In [22]:
df_out = pd.DataFrame(new_df_dict)
df_out

,ID,predictions_siam,predictions_ode,predicted_labels_siam,predicted_labels_ode,icp_full,abp_full
0,28901,"[10.52091, 1.276119, -10.414813, -4.50064, 2.0...","[2.1189032, 0.5201285, -3.07572, 1.3098269, -0...",0,0,"[0.052335985, 0.05443514, 0.059692558, 0.06839...","[0.0018919747089967132, 0.0, 0.007858305238187..."
1,9492,"[8.866835, 2.6159902, -8.96805, -3.0791683, 0....","[5.6902847, 1.5739464, -6.6032076, -0.97978276...",0,0,"[0.0, 0.0035480289, 0.012196649, 0.026370129, ...","[0.0, 0.0110935652628541, 0.010119953192770481..."
2,22264,"[5.5004606, 2.3892236, -5.825347, -0.89387333,...","[1.4549569, 1.1738801, -2.3937435, 1.0811548, ...",0,0,"[0.011769183, 0.015523151, 0.023788707, 0.0368...","[0.0021603847853839397, 0.0015785943251103163,..."
3,52245,"[2.4213831, -0.626239, -3.298091, 2.336095, -0...","[-2.8444102, -0.5259298, 1.2302781, 2.4965649,...",0,3,"[0.114803396, 0.116291694, 0.12007079, 0.12621...","[0.0023214893881231546, 0.0, 0.002903210930526..."
4,126772,"[2.264788, 0.62098056, -3.5394413, 0.80745363,...","[1.8539884, -0.24944848, -2.9861128, 0.6586227...",0,0,"[0.0, 0.0028781297, 0.009443885, 0.019976636, ...","[0.009257316589355469, 0.008544440381228924, 0..."
...,...,...,...,...,...,...,...
545,267427,"[7.492627, -7.8505926, -9.321978, -14.261319, ...","[-4.057676, 0.028914243, 2.667132, -1.0295558,...",4,2,"[0.0, 0.0016142861, 0.0054595894, 0.01176916, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
546,524837,"[3.3481314, -2.252114, -5.508824, -8.671974, 1...","[-3.3116314, -0.6420702, 1.7729827, 2.3051453,...",4,3,"[0.0, 0.0022855012, 0.0061536757, 0.01167377, ...","[0.021800952032208443, 0.022640826180577278, 0..."
547,218550,"[-0.3257271, -7.1318407, -2.6330853, -3.506983...","[-3.8048131, -1.8481992, 2.0302894, -1.0963886...",4,4,"[0.0, 0.0021298954, 0.0073015373, 0.0153130535...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
548,531958,"[7.0555134, -6.7449512, -9.390101, -15.944277,...","[-4.793749, -1.803917, 3.3353326, -0.69998807,...",4,4,"[0.0, 0.0015991221, 0.012775643, 0.0326188, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [23]:
df_shuff = df_out.sample(frac=1)
df_shuff = df_shuff.set_index(pd.Index([_ for _ in range(len(df_shuff))]))
df_shuff

,ID,predictions_siam,predictions_ode,predicted_labels_siam,predicted_labels_ode,icp_full,abp_full
0,75230,"[-1.5200852, 5.1051874, -0.18175413, -0.024464...","[-3.7580006, 0.9296253, 2.1666515, 2.1759653, ...",1,3,"[0.096139655, 0.09698825, 0.09998356, 0.105307...","[0.0061650266870856285, 0.005325749516487122, ..."
1,783477,"[-2.178629, 3.1389189, 0.39211398, 2.0207636, ...","[-4.8328648, -0.8775543, 3.1311605, 4.012851, ...",1,3,"[0.035204995, 0.036235593, 0.039316624, 0.0445...","[0.016612840816378593, 0.013853911310434341, 0..."
2,196385,"[-3.6899533, 5.517263, 1.4957453, -0.057287335...","[-4.511737, 1.5020214, 2.94979, 0.27513543, -0...",1,2,"[0.0, 0.00029428475, 0.005465972, 0.015859725,...","[0.01680969074368477, 0.011336625553667545, 0...."
3,654439,"[-4.458016, -1.5167022, 2.0830796, 5.2503486, ...","[-5.7176886, -1.4811232, 3.9196546, 4.438501, ...",3,3,"[0.044012174, 0.0456909, 0.05093992, 0.0601268...","[0.006194422487169504, 0.0027190186083316803, ..."
4,381518,"[-4.3558636, 1.7666216, 1.6224059, 2.6029673, ...","[-4.702006, 1.0715779, 3.0150402, 1.8787348, -...",3,2,"[0.0, 0.0023953344, 0.009100843, 0.020559898, ...","[0.0004678588593378663, 0.0011292756535112858,..."
...,...,...,...,...,...,...,...
545,583886,"[-2.2548537, 3.223869, 0.4646818, 2.0959663, -...","[-3.2986233, 0.37821382, 1.9225038, 3.1500542,...",1,3,"[0.0, 0.00073120894, 0.00406007, 0.01038905, 0...","[0.012628385797142982, 0.010349965654313564, 0..."
546,562609,"[-0.68753225, 5.094713, -1.0985967, -0.9541975...","[-3.4229152, 3.9631925, 1.4840955, -1.7176542,...",1,1,"[0.050730143, 0.0574469, 0.07189634, 0.0944087...","[0.0, 0.014564333483576775, 0.0133400931954383..."
547,530308,"[9.587189, -9.579541, -12.178752, -20.205217, ...","[0.8347183, -2.6403227, -1.9123682, -4.1929784...",4,4,"[0.012713513, 0.034923237, 0.07678108, 0.13591...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
548,287053,"[8.287413, -0.5612249, -8.870721, -4.9339004, ...","[4.084062, -0.25732133, -5.264055, -0.65195876...",0,0,"[0.0, 0.0017948775, 0.008963071, 0.02135202, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [24]:
df_shuff.to_csv(os.path.join(os.getcwd(), "results", "shuffled_additional_patients_check.csv"))

In [25]:
save_path = os.path.join(os.getcwd(), "results", "additional_experts_check")
for i in range(len(df_shuff)):
    item = df_shuff.iloc[i]
    plt.plot(item["icp_full"], label="ICP")
    plt.plot(item["abp_full"], label="ABP")
    plt.legend(loc="upper left")
    plt.savefig(os.path.join(save_path, str(i)+'.png'))
    plt.clf()

<Figure size 432x288 with 0 Axes>

In [2]:
df_work = pd.read_csv(os.path.join(os.getcwd(), "results", "shuffled_new_patients_check.csv"))

In [5]:
df_work = df_work.set_index(df_work.columns[0])

In [6]:
df_work

,ID,predictions_siam,predictions_ode,predicted_labels_siam,predicted_labels_ode,icp_full,abp_full
Unnamed: 0,,,,,,,
0,840218,[-10.280146 -4.354658 5.871772 7.30234...,[-7.273423 -1.0846035 4.978949 4.0313168 -...,3,2,"[0.07153258, 0.07192248, 0.07504449, 0.0811373...",[0. 0. 0. 0. 0...
1,644440,[-4.7873554 -1.2618821 2.236966 4.653909...,[-6.0355635 -1.7164685 4.197522 4.4738965 -...,3,3,"[0.06240144, 0.06416295, 0.07040156, 0.0815037...",[5.62788919e-04 0.00000000e+00 3.91062058e-04 ...
2,412620,[-15.998931 -0.8508374 9.785589 5.05396...,[-6.982583 -1.1597848 5.0000715 3.4071004 -...,2,2,"[0.08154324, 0.08473039, 0.09207965, 0.1038286...",[0.00274593 0.00217622 0.01040705 0.02587048 0...
3,733977,[-1.9037917 2.7162507 0.15208611 3.005653...,[-4.8704915 -0.6190575 3.2096121 3.8258505 -...,3,3,"[0.03825461, 0.03920087, 0.04250675, 0.0483136...",[0.00141168 0.00302805 0.00310579 0. 0...
4,729818,[-2.03402 -1.461345 0.1334122 5.85446 -...,[-4.046963 -1.1396823 2.575238 4.0721836 -...,3,3,"[0.01954702, 0.01931953, 0.02160951, 0.0267124...",[0.01183282 0.00792943 0.0079044 0.00355155 0...
...,...,...,...,...,...,...,...
145,262539,[-0.4146085 -4.3376336 -2.2976124 -3.5047154 1...,[-6.975032 -2.0750928 4.8661137 4.889391...,4,3,"[0.00141573651, 0.000430659187, 0.0, 0.0001211...",[1.12237968e-03 2.24721315e-03 3.70404217e-03 ...
146,141401,[-2.4265356 2.6200578 0.49974763 2.222854...,[ 1.6773369 0.2544927 -2.612067 1.058374...,1,0,"[0.04760507, 0.0478486, 0.04925613, 0.05208389...",[0.01683606 0.01693947 0.01412731 0.01245789 0...
147,581400,[-2.1944919 0.4988215 0.350736 4.412537 -...,[-2.1362083 0.03811647 0.87220085 2.721765...,3,3,"[0.0, 0.00178199, 0.00698641, 0.0161224, 0.029...",[6.31768256e-03 5.73656289e-03 1.85325521e-03 ...


In [10]:
icps = []
abps = []
for row in df_work.iterrows():
    row = row[1]
    icp = row["icp_full"][1:-1].split(",")
    icp = [float(value.strip("\n")) for value in icp if value is not ""]
    icps.append(icp)
    abp = row["abp_full"][1:-1].split(" ")
    abp = [float(value.strip("\n")) for value in abp if value is not ""]
    abps.append(abp)
ICP_full = torch.tensor(icps ,dtype=torch.float)
ABP_full = torch.tensor(abps,dtype=torch.float)

In [14]:
from models.SiameseModels import SiameseNeuralODE, SiameseResNet
from models.ResnetODEmodels import ODE, ResNet

class_dict = {
    0: "T1",
    1: "T2",
    2: "T3",
    3: "T4",
    4: "A+E"
}

name = "Resampling_SiameseODE_5cls_2020-06-16_SiameseODE_1"
model_path = os.path.join(os.getcwd(), "experiments", name, "model_1.pth")

device = torch.device('cuda:' + str(0) if torch.cuda.is_available() else 'cpu')
model_siam = SiameseNeuralODE(len(class_dict.keys())).to(device)
model_siam.load_state_dict(torch.load(model_path, map_location=device)["state_dict"])
model_siam.eval()

name = "2020-06-11_ODE_5cls_1"
model_path = os.path.join(os.getcwd(), "experiments", name, "model_1.pth")

model_ode = ODE(len(class_dict.keys())).to(device)

state_dict = torch.load(model_path, map_location=device)["state_dict"]
new_state_dict = {}
for key in state_dict.keys():
    new_state_dict["model."+key] = state_dict[key]

model_ode.load_state_dict(new_state_dict)
model_ode.eval()

name = "Resampling_SiameseResNet_5cls_2020-06-13_SiameseResnet_1"
model_path = os.path.join(os.getcwd(), "final_models", name, "model_1.pth")

device = torch.device('cuda:' + str(0) if torch.cuda.is_available() else 'cpu')
model_siam_res = SiameseResNet(len(class_dict.keys())).to(device)
model_siam_res.load_state_dict(torch.load(model_path, map_location=device)["state_dict"])
model_siam_res.eval()

name = "2020-06-11_Resnet_5cls_1"
model_path = os.path.join(os.getcwd(), "final_models", name, "model_1.pth")

model_res = ResNet(len(class_dict.keys())).to(device)

state_dict = torch.load(model_path, map_location=device)["state_dict"]
new_state_dict = {}
for key in state_dict.keys():
    new_state_dict["model."+key] = state_dict[key]

model_res.load_state_dict(new_state_dict)
model_res.eval()

ResNet(
  (model): Sequential(
    (0): Conv1d(1, 64, kernel_size=(3,), stride=(1,))
    (1): ResBlock(
      (norm1): GroupNorm(32, 64, eps=1e-05, affine=True)
      (relu): ReLU(inplace=True)
      (downsample): Conv1d(64, 64, kernel_size=(1,), stride=(2,), bias=False)
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(2,), padding=(1,), bias=False)
      (norm2): GroupNorm(32, 64, eps=1e-05, affine=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    )
    (2): ResBlock(
      (norm1): GroupNorm(32, 64, eps=1e-05, affine=True)
      (relu): ReLU(inplace=True)
      (downsample): Conv1d(64, 64, kernel_size=(1,), stride=(2,), bias=False)
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(2,), padding=(1,), bias=False)
      (norm2): GroupNorm(32, 64, eps=1e-05, affine=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    )
    (3): ResBlock(
      (norm1): GroupNorm(32, 64, eps=1e-05, affi

In [15]:
icp = ICP_full.to(device)
abp = ABP_full.to(device)
    
prediction_siam = model_siam(icp, abp)
prediction_siam_res = model_siam_res(icp, abp)
icp = icp.unsqueeze(1)
prediction_ode = model_ode(icp)
prediction_res = model_res(icp)

predicted_siam = list(np.argmax(prediction_siam.cpu().detach().numpy(), axis=1))
predicted_ode = list(np.argmax(prediction_ode.cpu().detach().numpy(), axis=1))
predicted_siam_res = list(np.argmax(prediction_siam_res.cpu().detach().numpy(), axis=1))
predicted_res = list(np.argmax(prediction_res.cpu().detach().numpy(), axis=1))

In [21]:
experts_dataframe = pd.read_csv(os.path.join(os.getcwd(), "results", "Final_check.csv"))
experts_dataframe = experts_dataframe.set_index(experts_dataframe.columns[0])

In [22]:
experts_dataframe

,FINAL TYPE
PULSE,
0,3
1,3
2,3
3,4
4,4
...,...
145,5
146,1
147,4


In [29]:
df_final = df_work.drop(columns = ["predictions_siam", "predictions_ode", "predicted_labels_siam", "predicted_labels_ode"])

In [30]:
df_final = df_final.join(experts_dataframe)

In [31]:
df_final

,ID,icp_full,abp_full,FINAL TYPE
Unnamed: 0,,,,
0,840218,"[0.07153258, 0.07192248, 0.07504449, 0.0811373...",[0. 0. 0. 0. 0...,3
1,644440,"[0.06240144, 0.06416295, 0.07040156, 0.0815037...",[5.62788919e-04 0.00000000e+00 3.91062058e-04 ...,3
2,412620,"[0.08154324, 0.08473039, 0.09207965, 0.1038286...",[0.00274593 0.00217622 0.01040705 0.02587048 0...,3
3,733977,"[0.03825461, 0.03920087, 0.04250675, 0.0483136...",[0.00141168 0.00302805 0.00310579 0. 0...,4
4,729818,"[0.01954702, 0.01931953, 0.02160951, 0.0267124...",[0.01183282 0.00792943 0.0079044 0.00355155 0...,4
...,...,...,...,...
145,262539,"[0.00141573651, 0.000430659187, 0.0, 0.0001211...",[1.12237968e-03 2.24721315e-03 3.70404217e-03 ...,5
146,141401,"[0.04760507, 0.0478486, 0.04925613, 0.05208389...",[0.01683606 0.01693947 0.01412731 0.01245789 0...,1
147,581400,"[0.0, 0.00178199, 0.00698641, 0.0161224, 0.029...",[6.31768256e-03 5.73656289e-03 1.85325521e-03 ...,4


In [35]:
new_df = {
    "predictions_siam_ode" : [i+1 for i in predicted_siam],
    "predictions_ode" : [i+1 for i in predicted_ode],
    "predictions_siam_resnet" : [i+1 for i in predicted_siam_res],
    "predictions_resnet" : [i+1 for i in predicted_res]
}
new_df = pd.DataFrame(new_df)
new_df

,predictions_siam_ode,predictions_ode,predictions_siam_resnet,predictions_resnet
0,4,3,4,3
1,4,4,3,4
2,3,3,3,3
3,4,4,4,4
4,4,4,4,4
...,...,...,...,...
145,5,4,5,4
146,2,1,1,1
147,4,4,4,4
148,3,2,3,3


In [36]:
df_final = df_final.join(new_df)

In [39]:
df_final = df_final.rename(columns={"FINAL TYPE": "final_type"})
df_final

,ID,icp_full,abp_full,final_type,predictions_siam_ode,predictions_ode,predictions_siam_resnet,predictions_resnet
Unnamed: 0,,,,,,,,
0,840218,"[0.07153258, 0.07192248, 0.07504449, 0.0811373...",[0. 0. 0. 0. 0...,3,4,3,4,3
1,644440,"[0.06240144, 0.06416295, 0.07040156, 0.0815037...",[5.62788919e-04 0.00000000e+00 3.91062058e-04 ...,3,4,4,3,4
2,412620,"[0.08154324, 0.08473039, 0.09207965, 0.1038286...",[0.00274593 0.00217622 0.01040705 0.02587048 0...,3,3,3,3,3
3,733977,"[0.03825461, 0.03920087, 0.04250675, 0.0483136...",[0.00141168 0.00302805 0.00310579 0. 0...,4,4,4,4,4
4,729818,"[0.01954702, 0.01931953, 0.02160951, 0.0267124...",[0.01183282 0.00792943 0.0079044 0.00355155 0...,4,4,4,4,4
...,...,...,...,...,...,...,...,...
145,262539,"[0.00141573651, 0.000430659187, 0.0, 0.0001211...",[1.12237968e-03 2.24721315e-03 3.70404217e-03 ...,5,5,4,5,4
146,141401,"[0.04760507, 0.0478486, 0.04925613, 0.05208389...",[0.01683606 0.01693947 0.01412731 0.01245789 0...,1,2,1,1,1
147,581400,"[0.0, 0.00178199, 0.00698641, 0.0161224, 0.029...",[6.31768256e-03 5.73656289e-03 1.85325521e-03 ...,4,4,4,4,4


In [41]:
all_examples = len(df_final)

right_ode = len(df_final.query("final_type == predictions_ode"))
right_resnet = len(df_final.query("final_type == predictions_resnet"))
right_siam_ode = len(df_final.query("final_type == predictions_siam_ode"))
right_siam_resnet = len(df_final.query("final_type == predictions_siam_resnet"))

In [44]:
print("Accuracy: \n ODE: {}%,\n ResNet : {}%,\n SiameseODE: {}%,\n SiameseResNet: {}%".format(100*right_ode/all_examples,
                                                                                              100*right_resnet/all_examples,
                                                                                              100*right_siam_ode/all_examples,
                                                                                              100*right_siam_resnet/all_examples,))

Accuracy: 
 ODE: 73.33333333333333%,
 ResNet : 70.66666666666667%,
 SiameseODE: 74.66666666666667%,
 SiameseResNet: 75.33333333333333%


In [47]:
from sklearn.metrics import classification_report
print(classification_report(df_final["final_type"], df_final["predictions_ode"], digits=3))

              precision    recall  f1-score   support

           1      0.950     0.655     0.776        29
           2      0.818     0.692     0.750        26
           3      0.690     0.784     0.734        37
           4      0.533     0.923     0.676        26
           5      0.952     0.625     0.755        32

    accuracy                          0.733       150
   macro avg      0.789     0.736     0.738       150
weighted avg      0.791     0.733     0.739       150



In [48]:
print(classification_report(df_final["final_type"], df_final["predictions_resnet"], digits=3))

              precision    recall  f1-score   support

           1      0.826     0.655     0.731        29
           2      0.947     0.692     0.800        26
           3      0.733     0.595     0.657        37
           4      0.473     1.000     0.642        26
           5      0.913     0.656     0.764        32

    accuracy                          0.707       150
   macro avg      0.779     0.720     0.719       150
weighted avg      0.782     0.707     0.716       150



In [49]:
print(classification_report(df_final["final_type"], df_final["predictions_siam_resnet"], digits=3))

              precision    recall  f1-score   support

           1      0.771     0.931     0.844        29
           2      0.645     0.769     0.702        26
           3      0.864     0.514     0.644        37
           4      0.750     0.808     0.778        26
           5      0.765     0.812     0.788        32

    accuracy                          0.753       150
   macro avg      0.759     0.767     0.751       150
weighted avg      0.767     0.753     0.747       150



In [50]:
print(classification_report(df_final["final_type"], df_final["predictions_siam_ode"], digits=3))

              precision    recall  f1-score   support

           1      0.767     0.793     0.780        29
           2      0.607     0.654     0.630        26
           3      0.833     0.676     0.746        37
           4      0.636     0.808     0.712        26
           5      0.897     0.812     0.852        32

    accuracy                          0.747       150
   macro avg      0.748     0.749     0.744       150
weighted avg      0.761     0.747     0.749       150



In [3]:
reverse_dict = {
    1: "T1",
    2: "T2",
    3: "T3",
    4: "T4",
    5: "A+E"
}
@interact(x=(0,149,1))
def plot(x=0):
    row = df_final.iloc[x]
    icp = row["icp_full"][1:-1].split(",")
    icp = [float(value.strip("\n")) for value in icp if value is not ""]
    abp = row["abp_full"][1:-1].split(" ")
    abp = [float(value.strip("\n")) for value in abp if value is not ""]
    plt.clf()
    plt.plot(icp, label="ICP")
    plt.plot(abp,label="ABP")
    print("Expert's label: {},\n ODE label: {}, \n ResNet label: {}, \n SiameseODE label: {},\n SiameseResNet label: {}".format(
    reverse_dict[row["final_type"]],
    reverse_dict[row["predictions_ode"]],
    reverse_dict[row["predictions_resnet"]],
    reverse_dict[row["predictions_siam_ode"]],
    reverse_dict[row["predictions_siam_resnet"]]))

interactive(children=(IntSlider(value=0, description='x', max=149), Output()), _dom_classes=('widget-interact'…

In [56]:
df_final.to_csv(os.path.join(os.getcwd(), "results", "Experts_full.csv"))

In [2]:
df_final = pd.read_csv(os.path.join(os.getcwd(), "results", "Experts_full.csv"))

In [3]:
df_work = pd.read_csv(os.path.join(os.getcwd(), "results", "shuffled_new_patients_check.csv"))

In [19]:
import random

reverse_dict = {
    1: "T1",
    2: "T2",
    3: "T3",
    4: "T4",
    5: "A+E"
}

def get_noise(max_amp=0.5, min_sines = 1, max_sines = 8, max_hz = 10, length=180):
    noise = np.zeros(length)
    no_of_sines = random.randint(min_sines, max_sines)
    for i in range(no_of_sines):
        x = np.linspace(-np.pi, np.pi, length)
        f = random.randint(1,max_hz)
        fi = random.random()
        noise += np.sin(f * x + fi)
    
    noise = noise-min(noise)
    noise = noise/max(noise)
    noise = max_amp * noise + 1
    return noise

def apply_noise(signal):
    noise = get_noise(length=len(signal))
    return (signal*noise)/max(signal*noise)


@interact(x=(0,149,1))
def plot(x=0):
    row = df_work.iloc[x]
    icp = row["icp_full"][1:-1].split(",")
    icp = [float(value.strip("\n")) for value in icp if value is not ""]
    abp = row["abp_full"][1:-1].split(" ")
    abp = [float(value.strip("\n")) for value in abp if value is not ""]
    plt.clf()
    plt.plot(icp, label="ICP")
    plt.plot(apply_noise(icp),label="ABP")

interactive(children=(IntSlider(value=0, description='x', max=149), Output()), _dom_classes=('widget-interact'…

In [5]:
df_work = pd.read_csv(os.path.join(os.getcwd(), "results", "shuffled_additional_patients_check.csv"))
save_path = os.path.join(os.getcwd(), "results", "additional_experts_check_csv")
i = 0
for row in df_work.iterrows():
    row = row[1]
    icp = row["icp_full"][1:-1].split(" ")
    icp = [float(value.strip("\n")) for value in icp if value is not ""]
    abp = row["abp_full"][1:-1].split(" ")
    abp = [float(value.strip("\n")) for value in abp if value is not ""]
    data_dict = {
        "ICP": icp,
        "ABP": abp
    }
    data_dict = pd.DataFrame(data_dict)
    data_dict.to_csv(os.path.join(save_path, str(i)+".csv"))
    i+=1